In [2]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline,BitsAndBytesConfig, LlamaForCausalLM, LlamaTokenizer, GenerationConfig
import transformers
import torch
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.document_loaders import PDFPlumberLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
import os
import streamlit as st
from llama_index import Document, LangchainEmbedding, SimpleDirectoryReader, ListIndex,VectorStoreIndex, StorageContext, GPTVectorStoreIndex, LLMPredictor, ServiceContext, load_index_from_storage
#from llama_index.vector_stores import DeepLakeVectorStore
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import base64
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
import torch.cuda as cuda

from langchain.agents.agent_toolkits import (VectorStoreInfo, create_vectorstore_agent, VectorStoreToolkit)

/home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.12) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [10]:

from huggingface_hub import notebook_login
notebook_login()
     


In [ ]:
model_id = 'meta-llama/Llama-2-70b-chat-hf'

In [8]:
tokenizer = LlamaTokenizer.from_pretrained(model_id)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tokenizer = LlamaTokenizer(model_id)                                                         │
│   2                                                                                              │
│                                                                                                  │
│ /home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/transformers/models/llama/t │
│ okenization_llama.py:144 in __init__                                                             │
│                                                                                                  │
│   141 │   │   self.add_bos_token = add_bos_token                                                 │
│   142 │   │   self.add_eos_token = add_eos_token                                                 │
│   143 │   │   self.sp_model = spm.SentencePieceProcessor(**self.sp_model_kwargs)                 │
│ ❱ 144 │   │   self.sp_model.Load(vocab_file)                                                     │
│   145 │                                                                                          │
│   146 │   def __getstate__(self):                                                                │
│   147 │   │   state = self.__dict__.copy()                                                       │
│                                                                                                  │
│ /home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/sentencepiece/__init__.py:9 │
│ 05 in Load                                                                                       │
│                                                                                                  │
│    902 │   │   raise RuntimeError('model_file and model_proto must be exclusive.')               │
│    903 │     if model_proto:                                                                     │
│    904 │   │   return self.LoadFromSerializedProto(model_proto)                                  │
│ ❱  905 │     return self.LoadFromFile(model_file)                                                │
│    906                                                                                           │
│    907                                                                                           │
│    908 # Register SentencePieceProcessor in _sentencepiece:                                      │
│                                                                                                  │
│ /home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/sentencepiece/__init__.py:3 │
│ 10 in LoadFromFile                                                                               │
│                                                                                                  │
│    307 │   │   return _sentencepiece.SentencePieceProcessor_serialized_model_proto(self)         │
│    308 │                                                                                         │
│    309 │   def LoadFromFile(self, arg):                                                          │
│ ❱  310 │   │   return _sentencepiece.SentencePieceProcessor_LoadFromFile(self, arg)              │
│    311 │                                                                                         │
│    312 │   def _EncodeAsIds(self, text, enable_sampling, nbest_size, alpha, add_bos, add_eos, r  │
│    313 │   │   return _sentencepiece.SentencePieceProcessor__EncodeAsIds(self, text, enable_sam  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: Not found: "meta-llama/Llama-2-70b-chat-hf": No such

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-70b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)
model.eval()
print(f"Model loaded on {device}")